In [1]:
import numpy as np
import pandas as pd
import gmaps as gm
import math
import os

In [2]:
#data as of 7/23/23
places = pd.read_csv("SCM_places_geocodio.csv")


In [3]:
#removing rows with no lat-long
acc = places[places['Accuracy Score'] == 0].index
places = places.drop(acc)
places = places.reset_index(drop=True)

In [4]:
#formatting data so its usable by gmaps
latlong = places[['Latitude','Longitude']]
places['transaction amount'] = pd.to_numeric(places['transaction amount'])
places['transaction amount'] = places['transaction amount'].div(10)

In [5]:
gm.configure(api_key=os.environ['GOOGLE_API_KEY'])
fig = gm.figure(map_type='ROADMAP')

#creating weighted heatmap layer
heatmap_layer = gm.heatmap_layer(latlong, weights=places['transaction amount'], max_intensity = 70, point_radius = 15)
fig.add_layer(heatmap_layer)



In [6]:
#creating ad radius around Skyland stadium
center = (41.136148,-74.704443)
circle = [(0,0)] * 90
#did some math by hand to get the following values for degrees per mile
lat_dpm = 360/24859.73
long_dpm = 360/19039.39
rad_m = 20
for i in range(0,360,4):
    lat = center[0] + math.cos(math.radians(i)) * lat_dpm  * rad_m
    long = center[1] + math.sin(math.radians(i)) * long_dpm  * rad_m
    circle[i//4] = (lat,long)

#creating marker and polygons for stadium and ad radius
SKY = gm.Marker(center,info_box_content="Skylands Stadium")
ad_rad = gm.Polygon(circle,stroke_color='blue',fill_color='blue')
drawing = gm.drawing_layer(features = [SKY,ad_rad],show_controls=False)
fig.add_layer(drawing)

In [7]:
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
#testing intesity and radius configs
#heatmap_layer.max_intensity = 70
#heatmap_layer.point_radius = 15